# PA2

In [46]:
import pandas as pd
import os
from nltk.stem import PorterStemmer

In [47]:
folder_path = "IRTM"
docs = []
for root, dirs, files in os.walk(folder_path):
    for file_name in files:
        file_path = os.path.join(root, file_name)
        with open(file_path, "r") as file:
            doc = file.read()
            docs.append(doc)

## q1
Construct a dictionary based on the terms extracted from the given documents.
- Record the document frequency of each term.
- Save your dictionary as a txt file (dictionary.txt).

In [48]:
docs_txt = " ".join(docs)
print(len(docs_txt))

3293992


### tokenize (pa1)

In [56]:
def tokenize(doc, stopword):
    # lowercase
    doc = doc.lower()
    # 去除特殊符號
    new_doc = ""
    for i in doc:
        if (i >= 'a' and i <= 'z') or (i == ' ') or (i == '-'):
            new_doc += i
    new_doc = new_doc.replace('-', " ")
    # 斷詞
    tokens = [i for i in new_doc.split(" ")]

    # Stopword Removal
    stop_tokens = [i for i in tokens if i not in stopword]
    # Stemming
    porter = PorterStemmer()
    stemming_tokens = [porter.stem(i) for i in stop_tokens]
    output_words = ' '.join(stemming_tokens)
    path = 'tokenized_words.txt'
    f = open(path, 'w')
    f.write(output_words)
    return stemming_tokens

In [65]:
with open("stopwords.txt", "r") as file:
    stopword = file.read()
tokenized_words = tokenize(docs_txt, stopword)

In [66]:
# 計算詞頻
word_dict = {}
for i in tokenized_words:
    if i in word_dict:
        word_dict[i] += 1
    else:
        word_dict[i] = 1
print(word_dict)

{'white': 345, 'hous': 653, 'also': 823, 'keep': 182, 'close': 380, 'watch': 132, 'yugoslavia': 370, 'opposit': 605, 'forc': 664, 'step': 182, 'pressur': 90, 'presid': 1923, 'slobodan': 171, 'milosev': 1258, 'work': 544, 'nbc': 140, 'jim': 106, 'maceda': 19, 'belgrad': 336, 'tonight': 218, 'serbia': 284, 'eve': 71, 'gener': 356, 'strike': 163, 'two': 1302, 'hour': 440, 'roadblock': 23, 'tast': 5, 'come': 403, 'tomorrow': 49, 'say': 1368, 'nationwid': 23, 'stoppag': 6, 'begin': 176, 'purpos': 23, 'man': 320, 'stole': 27, 'presidenti': 330, 'elect': 1078, 'peopl': 1332, 'dont': 341, 'accept': 156, 'victim': 154, 'hostag': 41, 'one': 1433, 'person': 149, 'time': 935, 'four': 400, 'year': 1480, 'ago': 257, 'hundr': 168, 'thousand': 241, 'serb': 165, 'march': 89, 'call': 614, 'well': 454, 'protest': 280, 'last': 885, 'three': 526, 'month': 382, 'surviv': 125, 'that': 236, 'real': 85, 'question': 343, 'power': 638, 'confront': 44, 'regim': 96, 'combin': 23, 'wealth': 9, 'get': 467, 'rid': 9,

In [103]:
# 將 dict 存成 txt 並 sort
df = pd.DataFrame(word_dict.items(), columns=['term', 'df'])

df_sorted = df.sort_values(by='df', ascending=True).reset_index(drop=True)
# 製作 index
df_sorted['t_index'] = df_sorted.index + 1
# reorder columns
df_final = df_sorted[['t_index', 'term', 'df']]

df_final.to_csv("dictionary.txt", index=False)

## q2
Transfer each document into a tf-idf unit vector
- Save it as a txt file (DocID.txt).